# RFV

**RFV** significa recência, frequência, valor e é utilizado para segmentação de clientes baseado no comportamento de compras dos clientes e agrupa eles em clusters parecidos. Utilizando esse tipo de agrupamento podemos realizar ações de marketing e CRM melhores direcionadas, ajudando assim na personalização do conteúdo e até a retenção de clientes.

Para cada cliente é preciso calcular cada uma das componentes abaixo:

- Recência (R): Quantidade de dias desde a última compra.
- Frequência (F): Quantidade total de compras no período.
- Valor (V): Total de dinheiro gasto nas compras do período.

E é isso que iremos fazer abaixo.



In [2]:
import numpy as np
import pandas as pd
from datetime import datetime


In [3]:
df_compras = pd.read_csv('dados.csv',
                        infer_datetime_format=True,
                        parse_dates=['DiaCompra'])
df_compras.head(20)

/tmp/ipykernel_9297/2724439615.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_compras = pd.read_csv('dados.csv',


,ID_cliente,CodigoCompra,DiaCompra,ValorTotal
0,12747,537215,2020-12-05,358.56
1,12747,538537,2020-12-13,347.71
2,12747,541677,2021-01-20,303.04
3,12747,545321,2021-03-01,310.78
4,12747,551992,2021-05-05,442.96
5,12747,554549,2021-05-25,328.35
6,12747,558265,2021-06-28,376.30
7,12747,563949,2021-08-22,301.70
8,12747,569397,2021-10-04,675.38
9,12747,577104,2021-11-17,312.73


In [4]:
df_compras.shape

(16127, 4)

In [5]:
df_compras.dtypes

ID_cliente               int64
CodigoCompra             int64
DiaCompra       datetime64[ns]
ValorTotal             float64
dtype: object

In [6]:
df_compras['DiaCompra'].min()

Timestamp('2020-12-01 00:00:00')

In [7]:
df_compras['DiaCompra'].max()

Timestamp('2021-12-09 00:00:00')

In [8]:
dia_atual = datetime(2021,12,9)
print(dia_atual)

2021-12-09 00:00:00


## Recência 


### Quantos dias faz que o cliente fez a sua última compra?

In [1]:
### Criando os segmentos
def recencia_class(x, r, q_dict):
    """Classifica como melhor o menor quartil 
       x = valor da linha,
       r = recencia,
       q_dict = quartil dicionario   
    """
    if x <= q_dict[r][0.25]:
        return 'A'
    elif x <= q_dict[r][0.50]:
        return 'B'
    elif x <= q_dict[r][0.75]:
        return 'C'
    else:
        return 'D'


def freq_val_class(x, fv, q_dict):
    """Classifica como melhor o maior quartil 
       x = valor da linha,
       fv = frequencia ou valor,
       q_dict = quartil dicionario   
    """
    if x <= q_dict[fv][0.25]:
        return 'D'
    elif x <= q_dict[fv][0.50]:
        return 'C'
    elif x <= q_dict[fv][0.75]:
        return 'B'
    else:
        return 'A'


df_compras = pd.read_csv('./input/dados.csv',
                         infer_datetime_format=True,
                         parse_dates=['DiaCompra'])

## Recência

####**Quantos dias faz que o cliente fez a sua última compra?**

df_recencia = df_compras.groupby(by='ID_cliente',
                                 as_index=False)['DiaCompra'].max()
df_recencia.columns = ['ID_cliente', 'DiaUltimaCompra']
print(df_recencia.head())

## dia_atual

dia_atual = df_compras['DiaCompra'].max()
print(dia_atual)

df_recencia['Recencia'] = df_recencia['DiaUltimaCompra'].apply(
    lambda x: (dia_atual - x).days)

print(df_recencia.head())

df_recencia.drop('DiaUltimaCompra', axis=1, inplace=True)

## Frequência

####**Quantas vezes cada cliente comprou com a gente?**

df_frequencia = df_compras[['ID_cliente', 'CodigoCompra'
                            ]].groupby('ID_cliente').count().reset_index()
df_frequencia.columns = ['ID_cliente', 'Frequencia']
print(df_frequencia.head())

## Valor

#### Quanto que cada cliente gastou no periodo?

df_valor = df_compras[['ID_cliente', 'ValorTotal'
                       ]].groupby('ID_cliente').sum().reset_index()
df_valor.columns = ['ID_cliente', 'Valor']
print(df_valor.head())

## Criando a tabela RFV

df_RF = df_recencia.merge(df_frequencia, on='ID_cliente')
print(df_RFV.head())

df_RFV = df_RF.merge(df_valor, on='ID_cliente')
df_RFV.set_index('ID_cliente', inplace=True)
print(df_RFV.head())

## Segmentação de clientes utilizando o RFV

### Um jeito de segmentar os clientes é criando quartis para cada componente do RFV, sendo que o melhor quartil é chamado de 'A', o segundo melhor quartil de 'B', o terceiro melhor de 'C' e o pior de 'D'. O melhor e o pior depende da componente. Po exemplo, quanto menor a recência melhor é o cliente (pois ele comprou com a gente tem pouco tempo) logo o menor quartil seria classificado como 'A', já pra componente frêquencia a lógica se inverte, ou seja, quanto maior a frêquencia do cliente comprar com a gente, melhor ele/a é, logo, o maior quartil recebe a letra 'A'.
### Se a gente tiver interessado em mais ou menos classes, basta a gente aumentar ou diminuir o número de quantils pra cada componente.

### Quartis para o RFV

quartis = df_RFV.quantile(q=[0.25, 0.5, 0.75])
print(quartis)

df_RFV['R_quartil'] = df_RFV['Recencia'].apply(recencia_class,
                                                args=('Recencia', quartis))
df_RFV['F_quartil'] = df_RFV['Frequencia'].apply(freq_val_class,
                                                  args=('Frequencia', quartis))
df_RFV['V_quartil'] = df_RFV['Valor'].apply(freq_val_class,
                                             args=('Valor', quartis))

print(df_RFV.head())

df_RFV['RFV_Score'] = (df_RFV.R_quartil + df_RFV.F_quartil +
                       df_RFV.V_quartil)
print(df_RFV.head())

df_RFV['RFV_Score'].value_counts()

df_RFV[df_RFV['RFV_Score'] == 'AAA'].sort_values('Valor',
                                                 ascending=False).head(10)

### Ações de marketing/CRM

dict_acoes = {
    'AAA':
    'Enviar cupons de desconto, Pedir para indicar nosso produto pra algum amigo, Ao lançar um novo produto enviar amostras grátis pra esses.',
    'DDD':
    'Churn! clientes que gastaram bem pouco e fizeram poucas compras, fazer nada',
    'DAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar',
    'CAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar'
}

df_RFV['acoes de marketing/crm'] = df_RFV['RFV_Score'].map(dict_acoes)

print(df_RFV.head())

df_RFV.to_excel('./output/RFV_.xlsx')

/home/sos/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_9297/3453579499.py:39: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_compras = pd.read_csv('./input/dados.csv',


FileNotFoundError: [Errno 2] No such file or directory: './input/dados.csv'